In [1]:
!pip3 install -r requirements.txt

  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 32.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 39.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 35.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 33.8 MB/s eta 0:00:00
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [2]:
import json
import os
from pathlib import Path
import asyncio

import duckdb
import requests
import nest_asyncio


In [3]:
#Asyncio versiopn, not working due to problems with aiohttp

nest_asyncio.apply()

async def fetch_movie_details(movie_name: str, api_key: str) -> dict:
    return await requests.get(f"https://www.omdbapi.com/?t={movie_name}&apikey=6a6418c2").json()
    

async def read_movies_details_async(movies, api_key):
    tasks = [asyncio.ensure_future(fetch_movie_details(movie, api_key)) for movie in movies]
    return await asyncio.gather(*tasks)

def read_movies_details(movies, api_key):
    return asyncio.run(read_movies_details_async(movies, api_key))


In [4]:
def fetch_movie_details(movie_name: str, api_key: str) -> dict:
    print(movie_name)
    url = f"https://www.omdbapi.com/?t={movie_name}&apikey=6a6418c2"
    response = requests.get(url).json()
    response["request_url"]=url.rsplit("&apikey=")[0]
    return response

def read_movies_details(movies, api_key: str) ->list[dict]:
    output = []
    for m in movies:
        response = fetch_movie_details(m, api_key)
        if response.get("Response")=="True":
            output.append(response)
    return output

In [7]:
def save_json_file(path: str, json_content: dict) -> None:
    output_file = Path(path)
    output_file.parent.mkdir(exist_ok=True, parents=True)
    with output_file.open('w') as f:
        json.dump(json_content, f, indent=4)


def filter_movies(movies: list[str], movies_existing: list[str]) -> list[str]:
    return list(set(movies) - set(movies_existing))



movies = [m[0] for m in duckdb.sql("SELECT DISTINCT title from 'data/landing_zone/revenues_per_day.csv'").fetchall()]
with open('data/landing_zone/movies_details.json', "r") as f:
    existing_movies = json.loads(f.read()) or []

movies_to_read = filter_movies(movies, [m['Title'] for m in existing_movies])
data = read_movies_details(movies_to_read, "6a6418c2")

save_json_file('data/landing_zone/movies_details.json', existing_movies+data)

The Last Shot
Alien Trespass
Film Stars Don't Die in Liverpool
Coach Carter
Fast Color
Petite Maman
Gabrielle
The Conjuring: The Devil Made Me Do It
Dear Zindagi
The Open Road
Shrek 2
Quu00e9 Leu00f3n
Booksmart
More Than Funny
Don 2
Lizzie
Doctor Who
Creed II
Moffie
Harry  Potter IMAX Marathon
Is Genesis History?2018 Re-release
Suburbicon
Baaghi 2
Looking for Comedy in the Muslim World
The Godfather Coda: The Death of Michael Corleone2020 Re-release
DCI 2017 Tour Premeire
The Matador
Leave No Trace
The Spirit
Maleficent: Mistress of Evil
The History Boys
Wake Up Sid
Prometheus
The Young and Prodigious T.S. Spivet
The Musketeer
The Big Ugly
The Dry Land
Khachaturian
South Pacific2018 Re-release
The Big Lebowski2018 Re-release
Heavy Water
War Horse
Christmas Jars
The Female Brain
A Beautiful Day in the Neighborhood
Finding Fanny
Ferdinand
Marlina the Murderer in Four Acts
Billy Lynn's Long Halftime Walk
We Don't Live Here Anymore
Desolation Center
San Andreas
Star Wars: Episode VII - The

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))